In [1]:
from API import *
api = Database_API()
recreate = 1
if recreate:
    api.query("DROP DATABASE IF EXISTS SOYBEAN")
    api.connect()
    api.create_database()
    api.populate_database()
    api.connect()

2023-12-22 12:33:26.812852: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-22 12:33:26.812899: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-22 12:33:26.813652: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-22 12:33:26.818944: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-22 12:33:27.639676: W tensorflow/compiler/tf2

In [2]:
api.plant_field([2], "Jersey City")


In [3]:
field_list = api.get("SELECT * FROM FIELD WHERE field_number = 2 AND PLANTATION_NAME = \"Jersey City\"", FIELD)

In [4]:

api.load_model("./lake/cv/model.json", "./lake/cv/model.h5")
images = api.generate_field_images(0.95, 10)
predicted_fraction = api.predict_fraction(images, field_list[0],  0.95)
predicted_total_cost = api.predict_total_cost(predicted_fraction)
url = api.push_to_azure(field_list[0], images)

In [5]:
url = api.push_to_azure(field_list[0], images)

In [6]:
# First harvest: no correction
api.harvest_soybeans("Jersey City", "New York", 2, url = url, prediction = predicted_total_cost)

inventory_list = api.get(f"SELECT * FROM INVENTORY WHERE site_name = \"New York\" AND plantation_name = \"Jersey City\" AND harvest_date = \"2023-12-22\"", INVENTORY)
api.start_batch(inventory_list[0], 0)

batch_list = api.get(f"SELECT * FROM BATCH WHERE site_name = \"New York\" AND plantation_name = \"Jersey City\" AND harvest_date = \"2023-12-22\" AND production_unit = 0 AND production_date = \"2023-12-22\"", BATCH)
actual_production_cost = 6.25 # corresponds to 100% yield
api.finish_batch(batch_list[0], production_cost = actual_production_cost, price = actual_production_cost*api.markup )

Harvest transaction succeeded
Start batch transaction succeeded

                        UPDATE BATCH
                        SET finished = TRUE
                        WHERE production_unit = 0 AND production_date = "2023-12-22" AND harvest_date = "2023-12-22" AND plantation_name = "Jersey City" AND field_number = 2 AND site_name = "New York";
                    

                    INSERT INTO STOCK (package_date, production_unit, production_date, harvest_date, plantation_name, field_number, site_name, production_cost, price) VALUES ('2023-12-22', 0, '2023-12-22', '2023-12-22', 'Jersey City', 2, 'New York', 6.25, 6.875000000000001)
                    
Finish batch transaction succeeded


In [7]:
api.plant_field([3], "Jersey City")


In [8]:
# Second harvest: correction on the first harvest
images = api.generate_field_images(0.3, 10)

field_list = api.get("SELECT * FROM FIELD WHERE field_number = 3 AND PLANTATION_NAME = \"Jersey City\"", FIELD)

predicted_fraction = api.predict_fraction(images, field_list[0], 0.3)
predicted_total_cost = api.predict_total_cost(predicted_fraction)

field_list = api.get("SELECT * FROM FIELD WHERE field_number = 3 AND PLANTATION_NAME = \"Jersey City\"", FIELD)
url = api.push_to_azure(field_list[0], images)



In [9]:
api.correct_price("Jersey City")

predicted cost: 20.833333333333332
price: 6.875
count: 1
predicted cost: 20.833333333333332
corrected price: 14.895833333333334


14.895833333333334